<a href="https://colab.research.google.com/github/ankazar/bk/blob/main/Pars_bet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from bs4 import BeautifulSoup  
import pandas as pd  
import requests 
import time 
import re 
from time import strftime
from time import gmtime
import json
from IPython.display import clear_output

x1 = '{''}'.format('https://www.marathonbet.ru/su/live/45356')
url = x1


while True:
  soup = BeautifulSoup(requests.get(url).text, 'html.parser')
  if len(soup.find_all(class_ = 'category-container collapsed')) != 0:
    url += '?ecids='
    for meta in soup.find_all(class_ = 'category-container collapsed'):
      url += meta.attrs['data-category-treeid'] + ','
    soup = BeautifulSoup(requests.get(url).text, 'html.parser')

  data = soup.find(class_='pure-u-1-1').find_all('table')

  df_ = pd.DataFrame()
  for n, df in enumerate(pd.read_html(str(data))):
    if len(df.index) == 0:
      #print(pd.read_html(str(data))[n-1][2][0])
      df_div = pd.DataFrame({'division':[pd.read_html(str(data))[n-1][2][0]]})
    if len(df.index) == 6 and len(df.columns) == 9:
      #print(pd.read_html(str(data))[n].iloc[0].to_frame().T)
      #df_bet = pd.DataFrame(pd.read_html(str(data))[n].iloc[0].to_frame().T)
      df_bet = df.iloc[0].to_frame().T
      df_ = df_.append(pd.concat([df_div, df_bet], axis=1))
      #print(df)
      #df = df.append(df_div)
    #print(n, len(df.index))  

  df_.columns = ['division', 'info', 'tree', '2', 'k1', 'k2', '5', '6', '7', '8']
  df_.index = [i for i in range(len(df_.index))]

  df_['period_number'] = df_['info'].apply(lambda x: x.count(',') + 1)

  df_d = {}
  d_l = []
  for row in df_.index:
    df_d = {}
    i_d = json.loads(df_.iloc[row]['tree'])
    #df_d['row'] = row
    df_d['treeId'] = i_d['treeId']
    df_d['mainScore'] = i_d['mainScore']
    df_d['total'] = abs(int(i_d['mainScore'].split(':')[0]) - int(i_d['mainScore'].split(':')[1]))
    df_d['time'] = strftime("%M:%S", gmtime(i_d['matchTime']['seconds']))
    #print(df_.iloc[row]['tree'])
    #print(row)
    d_l.append(df_d)

  clear_output(wait=True)
  time.sleep(3)
  display(pd.concat([df_[['division', 'k1', 'k2', 'period_number']], pd.DataFrame(d_l)], axis=1))
  time.sleep(3)